In [4]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup

parser = BeautifulSoup(website_url,'html.parser')
table = parser.find('table',{'class':'wikitable'})
#print(table.tr.text)
headers="Postcode,Borough,Neighbourhood"
table1=""
for tr in table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        if not tds.text.strip():
            row1=row1+"Not assigned,"        
        else:
            newText = tds.text.strip().replace(",", " ")
            row1=row1+newText+","                
    row1=row1[:-1:]+"\n"
    table1=table1+row1

In [5]:
file=open("toronto.csv","wb")
file.write(bytes(headers,encoding="ascii",errors="ignore"))
file.write(bytes(table1,encoding="ascii",errors="ignore"))

7136

In [6]:
import pandas as pd
df = pd.read_csv('toronto.csv')
df.columns=["Postcode","Borough","Neighbourhood"]
df=df[df['Borough']!="Not assigned"]
df = df.groupby(['Postcode','Borough'], sort=False).agg( ', '.join)
df['Neighbourhood'] = df['Neighbourhood'].apply(lambda x: df['Borough'] if x == "Not assigned" else x)
df=df.reset_index()

In [8]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns=['Postcode','Latitude','Longitude']

In [9]:
df_merge = pd.merge(df,df_geo,on='Postcode')
df_merge.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [15]:
#!pip install geopy
#!pip install folium

In [18]:
from geopy.geocoders import Nominatim
import folium

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [21]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

for lat, lng, borough, Neighbourhood in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough'], df_merge['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [23]:
CLIENT_ID = '5WNWMTQSJL5NSNM52PGX1U3DWOMZA34T5QJDEBZ3P343XGGD' 
CLIENT_SECRET = 'ZNLTBIRUOPZI10GRJYI2IG1LLNWB1JO4SFT4VLFXQKE4L0XK' 
VERSION = '20180604' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius=500
LIMIT=100

Your credentails:
CLIENT_ID: 5WNWMTQSJL5NSNM52PGX1U3DWOMZA34T5QJDEBZ3P343XGGD
CLIENT_SECRET:ZNLTBIRUOPZI10GRJYI2IG1LLNWB1JO4SFT4VLFXQKE4L0XK


In [26]:
import requests

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):        
                    
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
                
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_n = getNearbyVenues(names=df_merge['Neighbourhood'],latitudes=df_merge['Latitude'],longitudes=df_merge['Longitude'])
toronto_n.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [28]:
new_toronto = toronto_n.groupby('Neighbourhood').count()
new_toronto.head()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
Alderwood / Long Branch,11,11,11,11,11,11
Bathurst Manor / Wilson Heights / Downsview North,20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,23,23,23,23,23,23
